### RDD Tutorial

In [1]:
# Create a distributed list of numbers on the cluster
nums = sc.parallelize([1,2,3,4])

# Define a function that will be executed on each element
squared = nums.map(lambda x:x*x)

In [2]:
# Now execute the function
squared.collect()

[1, 4, 9, 16]

In [4]:
reduce_nums_sum = nums.reduce(lambda x,y:x+y)
print(reduce_nums_sum)

10
